In [1]:
import torch
import sklearn
import sys
from tqdm import tqdm_notebook as tqdm
import random
import numpy as np
from torch.utils.data import Dataset, Subset
from torch.utils.data import TensorDataset, DataLoader
import torch.backends.cudnn as cudnn
from torch import nn
import torch.optim as optim
from sklearn.metrics import confusion_matrix
from tqdm import tqdm_notebook
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
# make dataloader
def create_dataloader(dataset_name, transform, valid=False):
    test_dl, valid_dl = None, None

    if dataset_name == 'MNIST':
        train_datasets = datasets.FashionMNIST(root='../datasets', train=True, download=True, transform=transform)
        if valid:
            n_samples = len(train_datasets)
            train_size = int(n_samples * 0.8)
            subset1_indices = list(range(0,train_size))
            subset2_indices = list(range(train_size,n_samples))

            train_dataset = Subset(train_datasets, subset1_indices)
            valid_dataset = Subset(train_datasets, subset2_indices)

            train_dl = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
            valid_dl = DataLoader(valid_dataset, batch_size=VALID_BATCH_SIZE, shuffle=False)
        else:
            train_dl = DataLoader(train_datasets,   batch_size=TRAIN_BATCH_SIZE, shuffle=True)

        test_dataset = datasets.FashionMNIST(root='../datasets', train=False, download=True, transform=transform)
        test_dl = DataLoader(test_dataset, batch_size=TEST_BATCH_SIZE, shuffle=False)
    
    return train_dl, test_dl, valid_dl



In [ ]:
def myConv2D(data, num=True, kernel_size, edge, padding=0, stride=1):
   p = padding
   s = stride

    if num:
    else:
        conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride=s, padding=p, bias=True, padding_mode='zeros')

        y = conv(data)
    
    return y


In [ ]:
def myPooling(data, num=True, kernel_size)

In [ ]:
#Q1: 2D convolution

In [ ]:
#Q2: Edge detection

In [ ]:
#Q3: Padding

In [ ]:
#Q4: Stride

In [ ]:
#Q5: Pooling